<a href="https://colab.research.google.com/github/Aishwarya138/Synapse-Learning-period/blob/NLP-week-3/Tasksentiment_rating_rnn.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# <font color='#6629b2'>Predicting sentiment ratings with RNN using Keras</font>


## <font color='#6629b2'>Study Material</font>
- https://web.stanford.edu/~jurafsky/slp3/9.pdf
- https://www.youtube.com/watch?v=UNmqTiOnRfg
- https://www.youtube.com/watch?v=WCUNPb-5EYI
- https://www.youtube.com/watch?v=OuYtk9Ymut4

## <font color='#6629b2'>Dataset</font>

The Large Movie Review Datasetconsists of 50,000 movie reviews from [IMDB](http://www.imdb.com/). The ratings are on a 1-10 scale, but the dataset specifically contains "polarized" reviews: positive reviews with a rating of 7 or higher, and negative reviews with a rating of 4 or lower. There are an equal number of positive and negative reviews. 

You can download the dataset from: https://www.kaggle.com/lakshmi25npathi/imdb-dataset-of-50k-movie-reviews

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
'''Load the dataset into variable "reviews". You can truncate the dataset to keep a few hundred records if it's
    taking too long to process/train. Keep in mind, bigger the dataset, higher the accuracy score!
'''
import pandas 
reviews = pandas.read_csv("/content/drive/MyDrive/NLP_week3/week3/IMDB Dataset.csv", encoding='utf-8',nrows=5000)

reviews

,review,sentiment
0,One of the other reviewers has mentioned that ...,positive
1,A wonderful little production. <br /><br />The...,positive
2,I thought this was a wonderful way to spend ti...,positive
3,Basically there's a family where a little boy ...,negative
4,"Petter Mattei's ""Love in the Time of Money"" is...",positive
...,...,...
4995,An interesting slasher film with multiple susp...,negative
4996,i watched this series when it first came out i...,positive
4997,Once again Jet Li brings his charismatic prese...,positive
4998,"I rented this movie, after hearing Chris Gore ...",negative


In [3]:
#Split the data into train and split
from sklearn.model_selection import train_test_split

X = reviews["review"]
y = reviews["sentiment"]
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

In [4]:
# convert the above train test datasets into a pandas dataframe. You shoud have 4 dataframes.
# Name them train_reviews, test_reviews, train_sentiment, test_sentiment.
# In train_sentiment and test_sentiment, convert "positive" to 1 and "negative" to 0

train_reviews = X_train.to_frame()
test_reviews = X_test.to_frame()
train_sentiment = y_train.to_frame()
test_sentiment = y_test.to_frame()

train_sentiment[train_sentiment == "positive"] = 1
train_sentiment[train_sentiment == "negative"] = 0
test_sentiment[test_sentiment == "positive"] = 1
test_sentiment[test_sentiment == "negative"] = 0

test_sentiment

,sentiment
1501,0
2586,0
2653,0
1055,1
705,0
...,...
3563,1
1538,0
1837,0
2380,0


In [5]:
train_reviews.head()

,review
1840,This film was on last week and although at tha...
2115,This is a great off-the-wall romantic comedy a...
4437,This is the worst sequel on the face of the wo...
1146,I am sad to say that I disagree with other peo...
2486,I should have known when I looked at the box i...


## <font color='#6629b2'>Preparing the data</font>

###  <font color='#6629b2'>Tokenization</font>

The first preprocessing step is to tokenize each of the reviews into (lowercased) individual words, since the models will encode the reviews at the word level (rather than subword units like characters, for example). For this we'll use [spaCy](https://spacy.io/), which is a fast and extremely user-friendly library that performs various language processing tasks. Once you load a spaCy model for a particular language, you can provide any text as input to the model, e.g. encoder(text) and access its linguistic features.

In [6]:
'''Lowercase and tokenise all the reviews in train_reviews using spacy'''

import spacy
encoder = spacy.load('en_core_web_sm')

def text_to_tokens(text_seqs):
    #complete this function that lowers and tokenizes the reviews
    token_seqs = [[word.lower_ for word in encoder(text_seq)] for text_seq in text_seqs]
    return token_seqs 

train_reviews['Tokenized_Review'] = text_to_tokens(train_reviews['review'])
train_reviews[['review','Tokenized_Review']][:10]

,review,Tokenized_Review
1840,This film was on last week and although at tha...,"[this, film, was, on, last, week, and, althoug..."
2115,This is a great off-the-wall romantic comedy a...,"[this, is, a, great, off, -, the, -, wall, rom..."
4437,This is the worst sequel on the face of the wo...,"[this, is, the, worst, sequel, on, the, face, ..."
1146,I am sad to say that I disagree with other peo...,"[i, am, sad, to, say, that, i, disagree, with,..."
2486,I should have known when I looked at the box i...,"[i, should, have, known, when, i, looked, at, ..."
44,"This movie struck home for me. Being 29, I rem...","[this, movie, struck, home, for, me, ., being,..."
2154,I can't get this flick off my brain. It's defi...,"[i, ca, n't, get, this, flick, off, my, brain,..."
1526,Tashan - the title itself explains the nature ...,"[tashan, -, the, title, itself, explains, the,..."
4855,A top notch Columbo from beginning to end. I p...,"[a, top, notch, columbo, from, beginning, to, ..."
1378,"""Coconut Fred's Fruit Salad Island!"" is a hila...","["", coconut, fred, 's, fruit, salad, island, !..."


###  <font color='#6629b2'>Lexicon</font>

Then we need to assemble a lexicon (aka vocabulary) of words that the model needs to know. Each tokenized word in the reviews is added to the lexicon, and then each word is mapped to a numerical index that can be read by the model. Since large datasets may contain a huge number of unique words, it's common to filter all words occurring less than a certain number of times, and replace them with some generic &lt;UNK&gt; token. The min_freq parameter in the function below defines this threshold. When assigning the indices, the number 1 will represent unknown words. The number 0 will represent "empty" word slots, which is explained below. Therefore "real" words will have indices of 2 or higher.

In [7]:
'''Count tokens (words) in texts and add them to the lexicon'''

import pickle

def make_lexicon(token_seqs, min_freq=1, use_padding=False):
    # First, count how often each word appears in the text. Save this count in a dictionary called token_counts
    token_counts = {}
    for seq in token_seqs:
        for token in seq:
            if token in token_counts:
                token_counts[token] += 1
            else:
                token_counts[token] = 1
    

    # Then, assign each word to a numerical index, i.e save  all these words in a list. Filter words that occur less than or equal to min_freq times.
    
    lexicon = [token for token, count in token_counts.items() if count >= min_freq]
    
    #create a dictionary lexicon that maps each word to its index. Note that indexes will start from 2,  index 0 is saved for padding and index 1 for unknown words ('<UNK>')
    
    lexicon = {token:idx + 2 for idx,token in enumerate(lexicon)}
    
    lexicon[u'<UNK>'] = 1 # Unknown words are those that occur fewer than min_freq times
    lexicon_size = len(lexicon)
    print(lexicon)

    print("LEXICON SAMPLE ({} total items):".format(lexicon_size))
    print(dict(list(lexicon.items())[:20]))
    
    return lexicon

lexicon = make_lexicon(token_seqs=train_reviews['Tokenized_Review'], min_freq=1)

{'this': 2, 'film': 3, 'was': 4, 'on': 5, 'last': 6, 'week': 7, 'and': 8, 'although': 9, 'at': 10, 'that': 11, 'time': 12, 'of': 13, 'the': 14, 'day': 15, '(': 16, 'around': 17, '6': 18, 'pm': 19, ')': 20, 'quality': 21, 'movies': 22, 'is': 23, 'almost': 24, 'never': 25, 'good': 26, 'least': 27, 'here': 28, 'mexican': 29, 'tv': 30, ',': 31, 'i': 32, 'could': 33, "n't": 34, 'switch': 35, 'off': 36, '.': 37, 'story': 38, 'about': 39, 'madelene': 40, 'moore': 41, 'really': 42, 'touches': 43, 'you': 44, 'she': 45, 'does': 46, 'come': 47, 'across': 48, 'as': 49, 'a': 50, 'very': 51, 'sympathetic': 52, 'character': 53, 'first': 54, 'but': 55, 'seeing': 56, 'whole': 57, 'just': 58, 'want': 59, 'her': 60, 'so': 61, 'succeed': 62, 'leaves': 63, 'thinking': 64, 'think': 65, 'basically': 66, 'due': 67, 'to': 68, 'great': 69, 'acting': 70, 'brooke': 71, 'johnson': 72, 'had': 73, 'heard': 74, 'before': 75, 'nor': 76, 'knowingly': 77, 'seen': 78, 'in': 79, 'another': 80, 'compliments': 81, 'hope': 8

###  <font color='#6629b2'>From strings to numbers</font>

Once the lexicon is built, we can use it to transform each review from a list of string tokens into a list of numerical indices.

In [8]:
'''Convert each review from a list of tokens to a list of numbers (indices)'''

def tokens_to_idxs(token_seqs, lexicon): 
    #complete this function to return a list of indexed tokens 
    idx_seqs = [[lexicon[token] if token in lexicon else lexicon['<UNK>'] for token in token_seq] for token_seq in token_seqs]
    return idx_seqs

train_reviews['Review_Idxs'] = tokens_to_idxs(token_seqs=train_reviews['Tokenized_Review'], lexicon=lexicon)
                                   
train_reviews[['Tokenized_Review', 'Review_Idxs']][:10]

,Tokenized_Review,Review_Idxs
1840,"[this, film, was, on, last, week, and, althoug...","[2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 1..."
2115,"[this, is, a, great, off, -, the, -, wall, rom...","[2, 23, 50, 69, 36, 85, 14, 85, 86, 87, 88, 39..."
4437,"[this, is, the, worst, sequel, on, the, face, ...","[2, 23, 14, 147, 148, 5, 14, 149, 13, 14, 150,..."
1146,"[i, am, sad, to, say, that, i, disagree, with,...","[32, 204, 205, 68, 206, 11, 32, 207, 109, 208,..."
2486,"[i, should, have, known, when, i, looked, at, ...","[32, 270, 271, 272, 273, 32, 274, 10, 14, 275,..."
44,"[this, movie, struck, home, for, me, ., being,...","[2, 98, 472, 473, 157, 217, 37, 136, 474, 31, ..."
2154,"[i, ca, n't, get, this, flick, off, my, brain,...","[32, 421, 34, 105, 2, 529, 36, 304, 530, 37, 1..."
1526,"[tashan, -, the, title, itself, explains, the,...","[610, 85, 14, 611, 612, 613, 14, 614, 13, 14, ..."
4855,"[a, top, notch, columbo, from, beginning, to, ...","[50, 769, 770, 209, 119, 739, 68, 267, 37, 32,..."
1378,"["", coconut, fred, 's, fruit, salad, island, !...","[285, 791, 792, 124, 793, 794, 795, 331, 285, ..."


In [9]:
'''Encode reviews (train_reviews['Review_Idxs']) as bag-of-words vectors'''

import numpy 

def idx_seqs_to_bows(idx_seqs, matrix_length):
    #complete the function to return an array having bag-of-words vectors of the encoded reviews
    # hint: numpy.bincount()
    bow_seqs = numpy.array([numpy.bincount(numpy.array(idx_seq), minlength=matrix_length) for idx_seq in idx_seqs])
    return bow_seqs    


bow_train_reviews = idx_seqs_to_bows(train_reviews['Review_Idxs'], 
                                     matrix_length=len(lexicon) + 1) #add one to length for padding)

print("TRAIN INPUT:\n", bow_train_reviews)
print("SHAPE:", bow_train_reviews.shape, "\n")

#Showing an example mapping string words to counts
lexicon_lookup = {idx: lexicon_item for lexicon_item, idx in lexicon.items()}
lexicon_lookup[0] = ""
pandas.DataFrame([(lexicon_lookup[idx], count) for idx, count in enumerate(bow_train_reviews[0])], 
                 columns=['Word', 'Count'])

TRAIN INPUT:
 [[0 0 3 ... 0 0 0]
 [0 0 1 ... 0 0 0]
 [0 0 6 ... 0 0 0]
 ...
 [0 0 2 ... 0 0 0]
 [0 0 7 ... 1 1 1]
 [0 0 2 ... 0 0 0]]
SHAPE: (3500, 38769) 



,Word,Count
0,,0
1,<UNK>,0
2,this,3
3,film,5
4,was,2
...,...,...
38764,odious,0
38765,possess.<br,0
38766,***<br,0
38767,hell.<br,0


##  <font color='#6629b2'>Building a Recurrent Neural Network </font>



###  <font color='#6629b2'>Numerical lists to matrices</font>

The input representation for the RNN explicitly encodes the order of words in the review. We'll return to the lists of the word indices contained in train_reviews['Review_Idxs']. The input to the model will be these number sequences themselves. We need to put all the reviews in the training set into a single matrix, where each row is a review and each column is a word index in that sequence. This enables the model to process multiple sequences in parallel (batches) as opposed to one at a time. Using batches significantly speeds up training. However, each review has a different number of words, so we create a padded matrix equal to the length on the longest review in the training set. For all reviews with fewer words, we prepend the row with zeros representing an empty word position. This is why the number 0 was not assigned as a word index in the lexicon. We can tell Keras to ignore these zeros during training.

In [10]:
from tensorflow.keras.preprocessing.sequence import pad_sequences

def pad_idx_seqs(idx_seqs):
    
    #find the biggest review's length and save it in the variable below
    max_seq_len = max(len(i) for i in idx_seqs)
    
    #pad all these indexed reviews and return these padded sequences
    #HINT: use pad_sequences function by keras
    padded_idx = pad_sequences(idx_seqs, max_seq_len)
    
    return padded_idx

train_padded_idxs = pad_idx_seqs(train_reviews['Review_Idxs'])

print("TRAIN INPUT:\n", train_padded_idxs)
print("SHAPE:", train_padded_idxs.shape, "\n")

TRAIN INPUT:
 [[    0     0     0 ...     3    84    37]
 [    0     0     0 ...    94   118    37]
 [    0     0     0 ...   203   161    37]
 ...
 [    0     0     0 ...    14    98    37]
 [    0     0     0 ...   288   288   288]
 [    0     0     0 ... 14131 17797   331]]
SHAPE: (3500, 1698) 



###  <font color='#6629b2'>Model Layers</font>
The RNN will have four layers:

**1. Input**: The input layer takes in the matrix of word indices.

**2. Embedding**: A [layer](https://keras.io/layers/embeddings/) that converts integer word indices into distributed vector representations (embeddings), which were introduced above. The difference here is that rather than plugging in embeddings from a pretrained model as before, the word embeddings will be learned inside the model itself. Thus, the input to the model will be the word indices rather than their embeddings, and the embedding values will change as the model is trained. The mask_zero=True parameter in this layer indicates that values of 0 in the matrix (the padding) will be ignored by the model.

**3. GRU**: A [recurrent (GRU) hidden layer](https://keras.io/layers/recurrent/), the central component of the model. As it observes each word in the review, it integrates the word embedding representation with what it's observed so far to compute a representation (hidden state) of the review at that timepoint. There are a few architectures for this layer - we use the GRU variation, Keras also provides LSTM or just the simple vanilla recurrent layer (see the materials at the bottom for an explanation of the difference). This layer outputs the last hidden state of the sequence (i.e. the hidden representation of the review after its last word is observed).

**4. Dense**: An output [layer](https://keras.io/layers/core/#dense) that predicts the rating for the review based on its GRU representation given by the previous layer. It has one dimension that contains a continuous value (the rating). Add a proper activation function.

###  <font color='#6629b2'>Parameters</font>

Our function for creating the RNN takes the following parameters:

**n_input_nodes**: As with the standard bag-of-words MLP, this is the number of unique words in the lexicon, plus one to account for the padding represented by 0 values. This indicates the number of rows in the embedding layer, where each row corresponds to a word.

**n_embedding_nodes**: the number of dimensions (units) in the embedding layer, which can be freely defined. Here, it is set to 300.

**n_hidden_nodes**: the number of dimensions in the GRU hidden layer. Like the embedding layer, this can be freely chosen. Here, it is set to 500.

In [11]:
'''Create the model'''

import tensorflow as tf
from tensorflow.keras import layers
from tensorflow.keras import Input, Model
from tensorflow.keras.layers import Embedding
from tensorflow.keras.layers import GRU, Dense
from tensorflow.keras.layers import InputLayer

def create_rnn_model(n_input_nodes, n_embedding_nodes, n_hidden_nodes):
    
    #complete this function to create a model and compile it having the 4 layers listed above.
    #Note: Layer 1 -  Technically the shape of this layer is (batch_size, len(train_padded_idxs)).
    # However, both the batch size and the length of the input matrix can be inferred from the input at training time. 
    # The batch size is implicitly included in the shape of the input, so it does not need to 
    # be specified as a dimension of the input. None can be given as placeholder for the input matrix length.
    # By defining it as None, the model is flexible in accepting inputs with different lengths.
    input_layer = Input(shape=(None,))

    # Layer 2
    embedding_layer = Embedding(input_dim=n_input_nodes, output_dim=n_embedding_nodes, mask_zero=True)(input_layer)

    # Layer 3
    gru_hidden_layer = GRU(units=n_hidden_nodes)(embedding_layer)

    # Layer 4
    output_layer = Dense(units=1)(gru_hidden_layer)

    #Specify which layers are input and output, compile model with loss and optimization functions
    model = Model(input_layer, output_layer)
    model.compile(loss='mean_squared_error', optimizer='adam')
    return model

In [12]:
rnn_model = create_rnn_model(n_input_nodes=len(lexicon) + 1, n_embedding_nodes=300, n_hidden_nodes=500)

###  <font color='#6629b2'>Training</font>

In [13]:
'''
Train the rnn_model using the padded sequences and y=train_reviews['Rating'].
You need  to convert train_reviews['Rating'] to tensor before passing it as an argument
Hint: tf.convert_to_tensor
batch_size=20, epochs=5
'''
tensor_y = tf.convert_to_tensor(train_sentiment[['sentiment']].values, dtype=tf.int64)

rnn_model.fit(x=train_padded_idxs, y=tensor_y, batch_size=20, epochs=5)

Epoch 1/5
175/175 [==============================] - 567s 3s/step - loss: 0.2376
Epoch 2/5
175/175 [==============================] - 554s 3s/step - loss: 0.1220
Epoch 3/5
175/175 [==============================] - 549s 3s/step - loss: 0.0482
Epoch 4/5
175/175 [==============================] - 546s 3s/step - loss: 0.0127
Epoch 5/5
175/175 [==============================] - 550s 3s/step - loss: 0.0042


###  <font color='#6629b2'>Prediction</font>

In [14]:
'''Put test reviews in padded matrix just how we did for train_reviews'''
test_reviews['Review_Token'] = text_to_tokens(test_reviews['review'])
test_reviews['Review_Idx'] = tokens_to_idxs(token_seqs=test_reviews['Review_Token'], lexicon=lexicon)

test_padded_idxs = pad_idx_seqs(test_reviews['Review_Idx'])

print("TEST INPUT:\n", test_padded_idxs)
print("SHAPE:", test_padded_idxs.shape, "\n")

TEST INPUT:
 [[   0    0    0 ...  175 1630   37]
 [   0    0    0 ...   23  285 2510]
 [   0    0    0 ...  293  365  331]
 ...
 [   0    0    0 ...  352    1  331]
 [   0    0    0 ... 7000  820   37]
 [   0    0    0 ...  830  170    1]]
SHAPE: (1500, 1951) 



In [15]:
'''Predict the ratings '''

#Since ratings are integers, need to round predicted rating to nearest integer
test_reviews['RNN_Pred_Rating'] = numpy.round(rnn_model.predict(test_padded_idxs)[:,0]).astype(int)

###  <font color='#6629b2'>Evaluation</font>

In [16]:
'''Evaluate the model with R^2'''

# print the r2 score
from sklearn.metrics import r2_score 
r2 = r2_score(test_sentiment, test_reviews['RNN_Pred_Rating'])
print("R2 score is : ", r2)

R2 score is :  -0.039345506233401384


On the full test dataset of 25,000 reviews, the $R^2$ for this model is 0.622525. So the RNN outperforms the continuous bag-of-words MLP as well as the standard bag-of-words approach.
Your score might not be good because we're training on only 100-200 reviews.

### <font color='#6629b2'>Visualizing data inside the model</font>

To help visualize the data representation inside the model, we can look at the output of each layer in a model individually. Keras' Functional API lets you derive a new model with the layers from an existing model, so you can define the output to be a layer below the output layer in the original model. Calling predict() on this new model will produce the output of that layer for a given input. Of course, glancing at the numbers by themselves doesn't provide any interpretation of what the model has learned (although there are opportunities to [interpret these values](https://medium.com/civis-analytics/interpreting-and-visualizing-neural-networks-for-text-processing-e9dff0da9c22), but seeing them verifies the model is just a series of transformations from one matrix to another. The model stores its layers as the list model.layers, and you can retrieve specific layer by its position index in the model.

In [ ]:
'''Showing the output of the RNN embedding layer (second layer) for the test reviews'''

embedding_layer = Model(inputs=rnn_model.layers[0].input, 
                        outputs=rnn_model.layers[1].output) #embedding layer is 2nd layer (index 1)
embedding_output = embedding_layer.predict(test_padded_idxs)
print("EMBEDDING LAYER OUTPUT SHAPE:", embedding_output.shape)
print(embedding_output[0])

## <font color='#6629b2'>Conclusion</font>

As mentioned above, the models shown here could be applied to any task where the goal is to predict a score for a particular sequence. For ratings prediction, this output is ordinal, but it could also be categorical with a few simple changes to the output layer of the model. 

## <font color='#6629b2'>More resources</font>

Yoav Goldberg's book [Neural Network Methods for Natural Language Processing](http://www.morganclaypool.com/doi/abs/10.2200/S00762ED1V01Y201703HLT037) is a thorough introduction to neural networks for NLP tasks in general.

If you'd like to learn more about what Keras is doing under the hood, there is a [Theano tutorial](http://deeplearning.net/tutorial/lstm.html) that also applies an RNN to sentiment prediction, using the same dataset here

Andrej Karpathy's blog post [The Unreasonable Effectiveness of Recurrent Neural Networks](http://karpathy.github.io/2015/05/21/rnn-effectiveness/) is very helpful for understanding the mathematical details of an RNN, applied to the task of language modeling. It also provides raw Python code with an implementation of the backpropagation algorithm.

TensorFlow also has an RNN language model [tutorial](https://www.tensorflow.org/versions/r0.12/tutorials/recurrent/index.html) using the Penn Treebank dataset

Chris Olah provides a good [explanation](http://colah.github.io/posts/2015-08-Understanding-LSTMs/) of how LSTM RNNs work (this explanation also applies to the GRU model used here)

Denny Britz's [tutorial](http://www.wildml.com/2015/09/recurrent-neural-networks-tutorial-part-1-introduction-to-rnns/) documents well both the technical details of RNNs and their implementation in Python.